<a href="https://colab.research.google.com/github/Osondu-ifunanya/Urban-sprawl-impact-analysis-on-local-climate-using-remote-sensing-and-ML/blob/main/LstUrbanSprawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import PartialDependenceDisplay

# ---------------------------
# 1. Generate synthetic data
# ---------------------------
np.random.seed(42)
n_samples = 2000

# Features (synthetic, remote-sensing-like and socio-environmental)
# Urban fraction (0..1) — fraction of pixel that is built-up (key variable of interest)
urban_frac = np.random.beta(2, 5, n_samples)  # skewed toward low urban fraction

# NDVI (0..1) — vegetation indicator, inversely related to LST
ndvi = np.clip(np.random.normal(0.45 - 0.3 * urban_frac, 0.12, n_samples), 0, 1)

# Albedo (0..1) — surface reflectivity (urban tends to have lower albedo here for synthetic model)
albedo = np.clip(np.random.normal(0.2 + 0.1 * (1 - urban_frac), 0.05, n_samples), 0, 1)

# Elevation (m)
elevation = np.random.uniform(0, 800, n_samples)

# Population density (people/km2) — correlated with urban fraction
pop_density = np.round(np.random.normal(1000 * urban_frac + 50, 200), 0)
pop_density = np.clip(pop_density, 0, None)

# Distance to water (km) — near water tends to reduce LST modestly
dist_water = np.random.exponential(scale=5.0, size=n_samples)  # many near small distances

# Add a seasonal/temporal synthetic variable (e.g., midday solar index 0..1)
solar_index = np.random.uniform(0.6, 1.0, n_samples)

# Synthetic LST generation: define a plausible functional relationship with noise
# - Urban fraction increases LST
# - Higher NDVI reduces LST
# - Higher albedo reduces LST
# - Elevation slightly reduces LST (lapse rate)
# - Population density slightly increases LST
# - Distance to water increases LST (further -> hotter)
# - Solar index scales the effect
noise = np.random.normal(0, 1.2, n_samples)

lst = (
    25  # base temp
    + 6.0 * urban_frac * solar_index          # strong urban heat effect modulated by solar
    - 10.0 * ndvi                             # vegetation cooling
    - 8.0 * albedo                            # brighter surfaces cooler in this synthetic model
    - 0.0065 * elevation                      # lapse rate: 6.5°C per 1000 m
    + 0.002 * pop_density                     # small effect per person/km2
    + 0.3 * np.log1p(dist_water)              # distance-from-water effect
    + noise
)

# Build DataFrame
df = pd.DataFrame({
    'UrbanFrac': urban_frac,
    'NDVI': ndvi,
    'Albedo': albedo,
    'Elevation_m': elevation,
    'PopDensity': pop_density,
    'DistToWater_km': dist_water,
    'SolarIndex': solar_index,
    'LST': lst
})

# Quick look
print("First 5 rows of the synthetic dataset:")
print(df.head())

# ---------------------------------
# 2. Train/test split & model fit
# ---------------------------------
features = ['UrbanFrac', 'NDVI', 'Albedo', 'Elevation_m', 'PopDensity', 'DistToWater_km', 'SolarIndex']
X = df[features].values
y = df['LST'].values

X_train, X_test, y_train, y_test, df_train, df_test = train_test_split(X, y, df, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=200, random_state=42, min_samples_leaf=5)
model.fit(X_train, y_train)

# ---------------------------------
# 3. Evaluate model
# ---------------------------------
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"\nModel performance on test set: RMSE = {rmse:.3f} °C, R² = {r2:.3f}")

# Plot true vs predicted
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.5, s=20)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('True LST (°C)')
plt.ylabel('Predicted LST (°C)')
plt.title('True vs Predicted LST')
plt.grid(True)
plt.tight_layout()
plt.show()

# ---------------------------------
# 4. Feature importances
# ---------------------------------
importances = model.feature_importances_
importance_df = pd.DataFrame({'feature': features, 'importance': importances}).sort_values('importance', ascending=False)
print("\nFeature importances:")
print(importance_df)

plt.figure(figsize=(7,4))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.gca().invert_yaxis()
plt.title('Feature Importances (Random Forest)')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

# ---------------------------------
# 5. Partial dependence for UrbanFrac (visual interpretation)
# ---------------------------------
# Use sklearn's PartialDependenceDisplay (works with ensemble models)
fig, ax = plt.subplots(figsize=(6,4))
PartialDependenceDisplay.from_estimator(model, X_train, ['UrbanFrac'], feature_names=features, ax=ax)
plt.tight_layout()
plt.show()

# ---------------------------------
# 6. Scenario analysis: urban sprawl (increase UrbanFrac)
# ---------------------------------
# We'll simulate an "urban sprawl" scenario where UrbanFrac increases by +0.1 (clipped to 1.0)
# for every sample and compute delta LST = LST_sprawl - LST_base.

df_scenario = df.copy()
delta = 0.10  # +10 percentage points of urban cover (urban sprawl)
df_scenario['UrbanFrac_sprawl'] = np.clip(df_scenario['UrbanFrac'] + delta, 0, 1)

# Build X matrices for base and sprawl (keeping other features same)
X_base = df_scenario[features].values
features_sprawl = features.copy()
# replace UrbanFrac with UrbanFrac_sprawl for scenario predictions
X_sprawl = X_base.copy()
urban_col_idx = features.index('UrbanFrac')
X_sprawl[:, urban_col_idx] = df_scenario['UrbanFrac_sprawl'].values

# Predict LST for base and scenario
lst_base_pred = model.predict(X_base)
lst_sprawl_pred = model.predict(X_sprawl)
lst_delta = lst_sprawl_pred - lst_base_pred

# Attach results to DataFrame and compute statistics
df_scenario['LST_Base_Pred'] = lst_base_pred
df_scenario['LST_Sprawl_Pred'] = lst_sprawl_pred
df_scenario['LST_Delta'] = lst_delta

print("\nUrban sprawl scenario summary (+0.10 UrbanFrac):")
print(df_scenario[['UrbanFrac', 'UrbanFrac_sprawl', 'LST_Base_Pred', 'LST_Sprawl_Pred', 'LST_Delta']].head())

print(f"\nMean LST increase across all samples: {df_scenario['LST_Delta'].mean():.3f} °C")
print(f"Median LST increase: {df_scenario['LST_Delta'].median():.3f} °C")
print(f"Max LST increase: {df_scenario['LST_Delta'].max():.3f} °C")

# Plot distribution of LST changes
plt.figure(figsize=(7,4))
plt.hist(df_scenario['LST_Delta'], bins=40, color='tomato', edgecolor='k', alpha=0.8)
plt.xlabel('Predicted LST change (°C) after +0.10 UrbanFrac')
plt.ylabel('Count')
plt.title('Distribution of Predicted LST Increase due to Urban Sprawl')
plt.grid(True)
plt.tight_layout()
plt.show()

# ---------------------------------
# 7. Export scenario results to Excel
# ---------------------------------
output_path = "lst_urban_sprawl_scenario.xlsx"
df_scenario.to_excel(output_path, index=False)
print(f"\nScenario results exported to: {output_path}")

# ---------------------------------
# 8. Example: show top 10 locations with highest LST increase
# ---------------------------------
top10 = df_scenario.sort_values('LST_Delta', ascending=False).head(10)
print("\nTop 10 samples most impacted by urban sprawl (+0.10 UrbanFrac):")
print(top10[['UrbanFrac', 'UrbanFrac_sprawl', 'NDVI', 'Albedo', 'PopDensity', 'DistToWater_km', 'LST_Delta']])
